<a href="https://colab.research.google.com/github/statzenthusiast921/Personal-Projects/blob/main/Office_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
import numpy as np
import pandas as pd
#!pip install keras
from tensorflow import keras 
import random
import sqlite3
from keras.models import Sequential

from tensorflow.keras.layers import Input, Dropout, Dense, concatenate, Embedding
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
#!pip install np_utils
#!conda install keras -y
# from tensorflow.keras.utils import np_utils
# from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

import warnings
warnings.filterwarnings('ignore')


In [ ]:
import os
os.getcwd()


'/content'

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving the-office-lines.xlsx to the-office-lines.xlsx


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/

#%ls

/content/drive/My Drive


In [ ]:
#Changing database into text
df = pd.read_excel('/content/the-office-lines.xlsx')


df.speaker = df.speaker.astype(str)
df.line_text = df.line_text.astype(str)
df = df[["speaker","line_text"]]

df[:10]

,speaker,line_text
0,Michael,All right Jim. Your quarterlies look very good...
1,Jim,"Oh, I told you. I couldn't close it. So..."
2,Michael,So you've come to the master for guidance? Is ...
3,Jim,"Actually, you called me in here, but yeah."
4,Michael,"All right. Well, let me show you how it's done."
5,Michael,"[on the phone] Yes, I'd like to speak to your ..."
6,Michael,"I've, uh, I've been at Dunder Mifflin for 12 y..."
7,Pam,Well. I don't know.
8,Michael,"If you think she's cute now, you should have s..."
9,Pam,What?


In [ ]:
%%time
All_Office_Scripts = ''
last_seg = ''

for line in df.values:
#     print(line[0])
#     print(line[2])
    character = line[0].lower()
    dialogue = line[1].lower()
    script = character+": "+dialogue+" \n\n"
    All_Office_Scripts += script

print(All_Office_Scripts[:2000])



michael: all right jim. your quarterlies look very good. how are things at the library? 

jim: oh, i told you. i couldn't close it. so... 

michael: so you've come to the master for guidance? is this what you're saying, grasshopper? 

jim: actually, you called me in here, but yeah. 

michael: all right. well, let me show you how it's done. 

michael: [on the phone] yes, i'd like to speak to your office manager, please. yes, hello. this is michael scott. i am the regional manager of dunder mifflin paper products. just wanted to talk to you manager-a-manger. [quick cut scene] all right. done deal. thank you very much, sir. you're a gentleman and a scholar. oh, i'm sorry. ok. i'm sorry. my mistake. [hangs up] that was a woman i was talking to, so... she had a very low voice. probably a smoker, so... [clears throat] so that's the way it's done. 

michael: i've, uh, i've been at dunder mifflin for 12 years, the last four as regional manager. if you want to come through here... see we have t

In [ ]:
text_file = open("All_Office_Scripts.txt", "w")
text_file.write(All_Office_Scripts)
text_file.close()

In [ ]:
#Prep text for RNN
Text_Data = All_Office_Scripts.lower()

if len(Text_Data) > 500000:
    Text_Data = Text_Data[:500000]

charindex = list(set(Text_Data))
charindex.sort() 
print(charindex)

np.save("charindex.npy", charindex)

['\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
%%time
CHARS_SIZE = len(charindex)
SEQUENCE_LENGTH = 75
X_train = []
Y_train = []
for i in range(0, len(Text_Data)-SEQUENCE_LENGTH, 1 ): 
    X = Text_Data[i:i + SEQUENCE_LENGTH]
    Y = Text_Data[i + SEQUENCE_LENGTH]
    X_train.append([charindex.index(x) for x in X])
    Y_train.append(charindex.index(Y))

X_train = np.reshape(X_train, (len(X_train), SEQUENCE_LENGTH))

Y_train = keras.utils.to_categorical(Y_train)


CPU times: user 30.6 s, sys: 448 ms, total: 31 s
Wall time: 31 s


In [ ]:
#!pip install -U numpy==1.18.5
#!conda install python=3.6
#Create the model


def get_model():
    model = Sequential()
    inp = Input(shape=(SEQUENCE_LENGTH, ))
    x = Embedding(CHARS_SIZE, 75, trainable=False)(inp)
    x = CuDNNLSTM(512, return_sequences=True,)(x)
    x = CuDNNLSTM(512, return_sequences=True,)(x)
    x = CuDNNLSTM(512,)(x)
    x = Dense(256, activation="elu")(x)
    x = Dense(128, activation="elu")(x)
    outp = Dense(CHARS_SIZE, activation='softmax')(x)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'],
                 )

    return model

model = get_model()

model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 75)]              0         
                                                                 
 embedding (Embedding)       (None, 75, 75)            4125      
                                                                 
 cu_dnnlstm (CuDNNLSTM)      (None, 75, 512)           1206272   
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 75, 512)           2101248   
                                                                 
 cu_dnnlstm_2 (CuDNNLSTM)    (None, 512)               2101248   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [ ]:
#Checkpoints and Custom Callback
filepath="model_checkpoint.hdf5"

checkpoint = ModelCheckpoint(filepath,
                             monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

early = EarlyStopping(monitor="loss",
                      mode="min",
                      patience=1)


In [ ]:
class TextSample(Callback):

    def __init__(self):
        super(Callback, self).__init__() 

    def on_epoch_end(self, epoch, logs={}):
        pattern = X_train[700]
        outp = []
        seed = [charindex[x] for x in pattern]
        sample = 'TextSample:' +''.join(seed)+'|'
        for t in range(100):
            x = np.reshape(pattern, (1, len(pattern)))
            pred = self.model.predict(x)
            result = np.argmax(pred)
            outp.append(result)
            pattern = np.append(pattern,result)
            pattern = pattern[1:len(pattern)]
        outp = [charindex[x] for x in outp]
        outp = ''.join(outp)
        sample += outp
        print(sample)

textsample = TextSample()



In [ ]:
#!pwd
#%ls

In [ ]:
#Load model
#filepath='/content/drive/My Drive/model_checkpoint.hdf5'
# model = load_model(filepath)

In [ ]:
#Train the model
model_callbacks = [checkpoint, early, textsample]
model.fit(X_train, Y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          callbacks = model_callbacks)




Epoch 1/5
1953/1953 [==============================] - ETA: 0s - loss: 2.0150 - accuracy: 0.4300
Epoch 00001: loss improved from inf to 2.01502, saving model to model_checkpoint.hdf5
TextSample:ake. [hangs up] that was a woman i was talking to, so... she had a very low| and the best of the because that is a back of the because that is a back of the because that is a b
1953/1953 [==============================] - 807s 410ms/step - loss: 2.0150 - accuracy: 0.4300
Epoch 2/5
1953/1953 [==============================] - ETA: 0s - loss: 1.3280 - accuracy: 0.5981
Epoch 00002: loss improved from 2.01502 to 1.32804, saving model to model_checkpoint.hdf5
TextSample:ake. [hangs up] that was a woman i was talking to, so... she had a very low|er to start to the best of the best of the best of the best of the best of the best of the best of t
1953/1953 [==============================] - 792s 406ms/step - loss: 1.3280 - accuracy: 0.5981
Epoch 3/5
1953/1953 [==============================] - ETA: 0s -

In [ ]:
#Save the model
# model = load_model(filepath)
model.save_weights("full_train_weights.hdf5")
model.save("full_train_model.hdf5")


In [ ]:
%%time
TEXT_LENGTH  = 5000
LOOPBREAKER = 3
#Generating new scripts


x = np.random.randint(0, len(X_train)-1)
pattern = X_train[x]
outp = []
for t in range(TEXT_LENGTH):
    if t % 500 == 0:
        print("%"+str((t/TEXT_LENGTH)*100)+" done")
        
    x = np.reshape(pattern, (1, len(pattern)))
    pred = model.predict(x, verbose=0)
    result = np.argmax(pred)
    outp.append(result)
    pattern = np.append(pattern,result)
    pattern = pattern[1:len(pattern)]
    ####loopbreaker####
    if t % LOOPBREAKER == 0:
        pattern[np.random.randint(0, len(pattern)-10)] = np.random.randint(0, len(charindex)-1)

#Let's see the results
outp = [charindex[x] for x in outp]
outp = ''.join(outp)

print(outp)



#Save the text output
# f =  open("output_text_sample.txt","w")
# f.write(outp)
# f.close()

%0.0 done
%10.0 done
%20.0 done
%30.0 done
%40.0 done
%50.0 done
%60.0 done
%70.0 done
%80.0 done
%90.0 done
: i want to go to the time. 

dwight: i want to talk to you and then i was a good guy and good to the boat. 

michael: i want to go to the time. 

dwight: i want to talk to you and then i want to go to the time. 

michael: i want to go to the time. 

dwight: i want to talk to you and then i was a good guy and going to be a good time. 

michael: i want to go to the time. 

dwight: i want to talk to you and then i want to say that i was a good guy and good to the best dundie. 

michael: i want to go to the time. 

michael: i want to go to the time. 

dwight: i want to talk to you and then i was a good guy and going to get the fire guy. 

michael: i want to take the same time. 

michael: i want to go to the time. 

michael: i want to go to the guy who wants to go to the guy and going to get the best business school. 

michael: i want to go to the time. 

michael: i want to go to th